# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.13573


extracting tarball to tmp_2303.13573...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:463: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2303.13876... done.


/tmp/ipykernel_2074/2881150532.py:48: LatexWarning: 2303.13876 did not run properly
[Errno 2] No such file or directory: 'gs'
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2303.13980


extracting tarball to tmp_2303.13980...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13573-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13573) | **Orbital decay in an accreting and eclipsing 13.7 minute orbital period  binary with a luminous donor**  |
|| K. B. Burdge, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2023-03-27*|
|*Comments*| *13 pages, 7 figures, 2 tables, submitted to ApJL*|
|**Abstract**| We report the discovery of ZTF J0127+5258, a compact mass-transferring binary with an orbital period of 13.7 minutes. The system contains a white dwarf accretor, which likely originated as a post-common envelope carbon-oxygen (CO) white dwarf, and a warm donor ($T_{\rm eff,\,donor}= 16,400\pm1000\,\rm K$). The donor probably formed during a common envelope phase between the CO white dwarf and an evolving giant which left behind a helium star or helium white dwarf in a close orbit with the CO white dwarf. We measure gravitational wave-driven orbital inspiral with $\sim 35\sigma$ significance, which yields a joint constraint on the component masses and mass transfer rate. While the accretion disk in the system is dominated by ionized helium emission, the donor exhibits a mixture of hydrogen and helium absorption lines. Phase-resolved spectroscopy yields a donor radial-velocity semi-amplitude of $771\pm27\,\rm km\, s^{-1}$, and high-speed photometry reveals that the system is eclipsing. We detect a {\it Chandra} X-ray counterpart with $L_{X}\sim 3\times 10^{31}\,\rm erg\,s^{-1}$. Depending on the mass-transfer rate, the system will likely evolve into either a stably mass-transferring helium CV, merge to become an R Crb star, or explode as a Type Ia supernova in the next million years. We predict that the Laser Space Interferometer Antenna (LISA) will detect the source with a signal-to-noise ratio of $24\pm6$ after 4 years of observations. The system is the first \emph{LISA}-loud mass-transferring binary with an intrinsically luminous donor, a class of sources that provide the opportunity to leverage the synergy between optical and infrared time domain surveys, X-ray facilities, and gravitational-wave observatories to probe general relativity, accretion physics, and binary evolution. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13676-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13676) | **Resolved stellar population properties of PHANGS-MUSE galaxies**  |
|| I. Pessa, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>, <mark>K. Kreckel</mark>, <mark>F. Pinna</mark> |
|*Appeared on*| *2023-03-27*|
|*Comments*| *52 pages, 48 figures, accepted for publication in A&A*|
|**Abstract**| Analyzing resolved stellar populations across the disk of a galaxy can provide unique insights into how that galaxy assembled its stellar mass over its lifetime. Previous work at ~1 kpc resolution has already revealed common features in the mass buildup (e.g., inside-out growth of galaxies). However, even at approximate kpc scales, the stellar populations are blurred between the different galactic morphological structures such as spiral arms, bars and bulges. Here we present a detailed analysis of the spatially resolved star formation histories (SFHs) of 19 PHANGS-MUSE galaxies, at a spatial resolution of ~100 pc. We show that our sample of local galaxies exhibits predominantly negative radial gradients of stellar age and [Z/H], consistent with previous findings, and a radial structure that is primarily consistent with local star formation, and indicative of inside-out formation. In barred galaxies, we find flatter [Z/H] gradients along the semi-major axis of the bar than along the semi-minor axis, as is expected from the radial mixing of material along the bar. In general, the derived assembly histories of the galaxies in our sample tell a consistent story of inside-out growth, where low-mass galaxies assembled the majority of their stellar mass later in cosmic history than high-mass galaxies. We also show how stellar populations of different ages exhibit different kinematics, with younger stellar populations having lower velocity dispersions than older stellar populations at similar galactocentric distances, which we interpret as an imprint of the progressive dynamical heating of stellar populations as they age. Finally, we explore how the time-averaged star formation rate evolves with time, and how it varies across galactic disks. This analysis reveals a wide variation of the SFHs of galaxy centers and additionally shows that structural features become less pronounced with age. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13732-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13732) | **Thermal Emission from the Hot Jupiter WASP-103b in $J$ and $K_{\rm s}$  Bands**  |
|| Y. Shi, et al. -- incl., <mark>M. Zhai</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2023-03-27*|
|*Comments*| **|
|**Abstract**| Hot Jupiters, particularly those with temperature higher than 2000\,K are the best sample of planets that allow in-depth characterization of their atmospheres. We present here a thermal emission study of the ultra hot Jupiter WASP\mbox{-}103\,b observed in two secondary eclipses with CFHT/WIRCam in $J$ and $K_{\rm s}$ bands. By means of high precision differential photometry, we determine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and 270\,ppm, which are combined with the published HST/WFC3 and Spitzer data to retrieve a joint constraints on the properties of WASP-103\,b dayside atmosphere. We find that the atmosphere is best fit with a thermal inversion layer included. The equilibrium chemistry retrieval indicates an enhanced C/O (1.35$^{+0.14}_{-0.17}$) and a super metallicity with [Fe/H]$=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity of WASP-103 of [Fe/H]=0.06, this planet seems to be $\sim$100 more abundant than its host star. The free chemistry retrieval analysis yields a large abundance of FeH, H$^{-}$, CO$_2$ and CH$_4$. Additional data of better accuracy from future observations of JWST should provide better constraint of the atmospheric properties of WASP-103b. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13980-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13980) | **Thermodynamic and Magnetic Topology Evolution of the X1.0 Flare on 2021  October 28 Simulated by a Data-driven Radiative Magnetohydrodynamic Model**  |
|| J.-h. Guo, et al. -- incl., <mark>Y. Guo</mark> |
|*Appeared on*| *2023-03-27*|
|*Comments*| *46 pages, 15 figures, accepted for publication in The Astrophysical Journal Supplement*|
|**Abstract**| Solar filament eruptions, flares and coronal mass ejections (CMEs) are manifestations of drastic release of energy in the magnetic field, which are related to many eruptive phenomena from the Earth magnetosphere to black hole accretion disks. With the availability of high-resolution magnetograms on the solar surface, observational data-based modelling is a promising way to quantitatively study the underlying physical mechanisms behind observations. By incorporating thermal conduction and radiation losses in the energy equation, we develop a new data-driven radiative magnetohydrodynamic (MHD) model, which has the capability to capture the thermodynamic evolution compared to our previous zero-\b{eta} model. Our numerical results reproduce major observational characteristics of the X1.0 flare on 2021 October 28 in NOAA active region (AR) 12887, including the morphology of the eruption, kinematic of flare ribbons, extreme-ultraviolet (EUV) radiations, and two components of the EUV waves predicted by the magnetic stretching model, i.e., a fast-mode shock wave and a slower apparent wave due to successive stretching of magnetic field lines. Moreover, some intriguing phenomena are revealed in the simulation. We find that flare ribbons separate initially and ultimately stop at the outer stationary quasi-separatrix layers (QSLs). Such outer QSLs correspond to the border of the filament channel and determine the final positions of flare ribbons, which can be used to predict the size and the lifetime of a flare before it occurs. In addition, the side view of the synthesized EUV and white-light images exhibit typical three-part structures of CMEs, where the bright leading front is roughly cospatial with the non-wave component of the EUV wave, reinforcing the magnetic stretching model for the slow component of EUV waves. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.13876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.13876) | **Borexino's search for low-energy neutrinos associated with gravitational  wave events from GWTC-3 database**  |
|| B. C. D. Basilico, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2023-03-27*|
|*Comments*| *13 pages, 8 figures*|
|**Abstract**| The search for neutrino events in correlation with gravitational wave (GW) events for three observing runs (O1, O2 and O3) from 09/2015 to 03/2020 has been performed using the Borexino data-set of the same period. We have searched for signals of neutrino-electron scattering with visible energies above 250 keV within a time window of 1000 s centered at the detection moment of a particular GW event. Two types of incoming neutrino spectra were considered: the mono-energetic line and the spectrum expected from supernovae. The same spectra were considered for electron antineutrinos detected through inverse beta-decay (IBD) reaction. GW candidates originated by merging binaries of black holes (BHBH), neutron stars (NSNS) and neutron star and black hole (NSBH) were analysed separately. In total, follow-ups of 74 out of 93 gravitational waves reported in the GWTC-3 catalog were analyzed and no statistically significant excess over the background was observed. As a result, the strongest upper limits on GW-associated neutrino and antineutrino fluences for all flavors (\nu_e, \nu_\mu, \nu_\tau) have been obtained in the (0.5 - 5.0) MeV neutrino energy range. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.13573.md
exported in  _build/html/2303.13676.md
    + _build/html/tmp_2303.13676/./Figures/radial_LW_age_all.png
    + _build/html/tmp_2303.13676/./Figures_SSP_fitting/fig_maps_test_templates_E-MILES_CB07_E-MILES+G-D.png
    + _build/html/tmp_2303.13676/./Figures/sigma_age_at_fixed_radii_disk.jpg
exported in  _build/html/2303.13732.md
    + _build/html/tmp_2303.13732/./pic/model_103b.png
    + _build/html/tmp_2303.13732/./pic/rxb2_J.png
    + _build/html/tmp_2303.13732/./pic/rms-beta.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{\mathrm{M_{\odot}}}$
$\newcommand{\msunyr}{\msun{\rm yr}^{-1}}$
$\newcommand{\mhe}{M_{\mathrm{He}}}$
$\newcommand{\mwd}{M_{\mathrm{WD}}}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\mesa}{{\tt\string MESA}}$
$\newcommand{\kB}{k_{\mathrm{B}}}$
$\newcommand{\NA}{N_{\mathrm{A}}}$
$\newcommand{\She}{s_{\mathrm{c},\mathrm{He}}}$
$\newcommand{\taumdot}{\tau_{\dot{M}}}$
$\newcommand{\tauth}{\tau_{\mathrm{th}}}$
$\newcommand{\Jdotgr}{\dot{J}_{\mathrm{gr}}}$
$\newcommand{\Porb}{P_{\mathrm{orb}}}$
$\newcommand{\Pcrit}{P_{\mathrm{c}}}$
$\newcommand{\Etide}{\dot{E}_{\mathrm{tide}}}$
$\newcommand{\Eheat}{\dot{E}_{\mathrm{heat}}}$
$\newcommand{\Teff}{T_{\mathrm{eff}}}$
$\newcommand{\Rmin}{R_{\mathrm{min}}}$
$\newcommand{\RWD}{R_{\mathrm{WD}}}$</div>



<div id="title">

# Orbital decay in an accreting and eclipsing 13.7 minute orbital period binary with a luminous donor

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.13573-b31b1b.svg)](https://arxiv.org/abs/2303.13573)<mark>Appeared on: 2023-03-27</mark> - _13 pages, 7 figures, 2 tables, submitted to ApJL_

</div>
<div id="authors">

K. B. Burdge, et al. -- incl., <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of ZTF J0127+5258, a compact mass-transferring binary with an orbital period of 13.7 minutes. The system contains a white dwarf accretor, which likely originated as a post-common envelope carbon-oxygen (CO) white dwarf, and a warm donor ( $T_{\rm eff, donor}= 16,400\pm1000 \rm K$ ). The donor probably formed during a common envelope phase between the CO white dwarf and an evolving giant which left behind a helium star or helium white dwarf in a close orbit with the CO white dwarf. We measure gravitational wave-driven orbital inspiral with $\sim 35\sigma$ significance, which yields a joint constraint on the component masses and mass transfer rate. While the accretion disk in the system is dominated by ionized helium emission, the donor exhibits a mixture of hydrogen and helium absorption lines. Phase-resolved spectroscopy yields a donor radial-velocity semi-amplitude of $771\pm27 \rm km  s^{-1}$ , and high-speed photometry reveals that the system is eclipsing. We detect a $_ Chandra_$ X-ray counterpart with $L_{X}\sim 3\times 10^{31} \rm erg s^{-1}$ . Depending on the mass-transfer rate, the system will likely evolve into either a stably mass-transferring helium CV, merge to become an R Crb star, or explode as a Type Ia supernova in the next million years. We predict that the Laser Space Interferometer Antenna (LISA) will detect the source with a signal-to-noise ratio of $24\pm6$ after 4 years of observations. The system is the first $*LISA*$ -loud mass-transferring binary with an intrinsically luminous donor, a class of sources that provide the opportunity to leverage the synergy between optical and infrared time domain surveys, X-ray facilities, and gravitational-wave observatories to probe general relativity, accretion physics, and binary evolution.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.13573"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\hii}{\ion{H}{II}}$
$\newcommand{\hi}{\ion{H}{I}}$
$\newcommand{\cii}{\ion{C}{II}}$
$\newcommand{\nii}{[\ion{N}{II}]}$
$\newcommand{\sii}{[\ion{S}{II}]}$
$\newcommand{\oiii}{[\ion{O}{III}]}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Resolved stellar population properties of PHANGS-MUSE galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.13676-b31b1b.svg)](https://arxiv.org/abs/2303.13676)<mark>Appeared on: 2023-03-27</mark> - _52 pages, 48 figures, accepted for publication in A&A_

</div>
<div id="authors">

I. Pessa, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>, <mark>K. Kreckel</mark>, <mark>F. Pinna</mark>

</div>
<div id="abstract">

**Abstract:** Analyzing resolved stellar populations across the disk of a galaxy can provide unique insights into how that galaxy assembled its stellar mass over its lifetime.Previous work at $\sim$ 1 kpc resolution has already revealed common features in the mass buildup (e.g., inside-out growth of galaxies). However, even at approximate kpc scales, the stellar populations are blurred between the different galactic morphological structures such as spiral arms, bars and bulges.  Here we present a detailed analysis of the spatially resolved star formation histories (SFHs) of 19 PHANGS-MUSE galaxies, at a spatial resolution of $\sim100$ pc. We show that our sample of local galaxies exhibits predominantly negative radial gradients of stellar age and metallicity, consistent with previous findings, and a radial structure that is primarily consistent with local star formation, and indicative of inside-out formation. In barred galaxies, we find flatter metallicity gradients along the semi-major axis of the bar than along the semi-minor axis, as is expected from the radial mixing of material along the bar during infall. In general, the derived assembly histories of the galaxies in our sample tell a consistent story of inside-out growth, where low-mass galaxies assembled the majority of their stellar mass later in cosmic history than high-mass galaxies (also known as "downsizing"). We also show how stellar populations of different ages exhibit different kinematics. Specifically, we find that younger stellar populations have lower velocity dispersions than older stellar populations at similar galactocentric distances, which we interpret as an imprint of the progressive dynamical heating of stellar populations as they age. Finally, we explore how the time-averaged star formation rate evolves with time, and how it varies across galactic disks. This analysis reveals a wide variation of the SFHs of galaxy centers and additionally shows that structural features become less pronounced with age.

</div>

<div id="div_fig1">

<img src="tmp_2303.13676/./Figures/radial_LW_age_all.png" alt="Fig9" width="100%"/>

**Figure 9. -** Luminosity-weighted stellar age radial profiles for the galaxies in our sampleLuminosity-weighted stellar age radial profiles for the galaxies in our sample. Different colors indicate the radial profile measured across different environments, as indicated in the legend of the bottom-right panel. The black line shows the radial profile measured for the entire FoV (i.e., all environments together). The galactocentric distance is measured in units of $R_{25}$, in order to measure radial distance homogeneously across our sample. The value of $R_{25}$(kpc) of each galaxy is indicated in each corresponding panel. The solid gray line shows the best-fit linear gradient for each galaxy. (*fig:radial_LW_age*)

</div>
<div id="div_fig2">

<img src="tmp_2303.13676/./Figures_SSP_fitting/fig_maps_test_templates_E-MILES_CB07_E-MILES+G-D.png" alt="Fig35" width="100%"/>

**Figure 35. -** Age, metallicity, extinction and stellar mass surface density maps obtained for the central region of NGC 3351, with the four different sets of templates testedLuminosity-weighted age (top row), luminosity-weighted [Z/H](second row), stellar E(B-V) (third row), and stellar mass surface density (bottom row) maps derived for the central star-forming ring of NGC 3351. Each column shows the maps obtained using the set of templates indicated at the top of each column. The maps obtained using our fiducial set of templates are shown in the left column. The area outside the red ellipse in the top-left panel is used to probe disk stellar population properties, and the area between the red and the black ellipses is used to probe stellar population properties of the young star-forming ring (see text). (*fig:base_case_tests*)

</div>
<div id="div_fig3">

<img src="tmp_2303.13676/./Figures/sigma_age_at_fixed_radii_disk.jpg" alt="Fig24" width="100%"/>

**Figure 24. -** Mean stellar velocity dispersion measured at a galactocentric radius of 0.25-0.30 $R_{25}$ in the disk environment of all our sample galaxies for stellar populations with different luminosity-weighted ages.Mean stellar velocity dispersion ($\sigma_{*}$) measured at a galactocentric radius of 0.25-0.30 $R_{25}$ in the disk environment of all our sample galaxies for stellar populations with luminosity-weighted ages within the three different age bins defined in the main text. We note that for visualization purposes, the $x$-axis position of each data point is located at the edge of the age bin it represents. In each panel, the leftmost triangle represents the mean (area-weighted) velocity dispersion of all populations younger than $100$ Myr. The central square marks the mean velocity dispersion of populations with 100 Myr < LumW age < 300 Myr, and the rightmost triangle shows the same quantity for populations older than 3 Gyr. The errorbar indicates the standard deviation of the $\sigma_{*}$ values of the pixels within a given age bin. The bottom right panel shows the trend for all galaxies, colored by total stellar mass. (*fig:sigma_age_fixed_radii*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.13676"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ww}[1]{{\textcolor{black}{#1}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# THERMAL EMISSION FROM THE HOT JUPITER WASP-103 b IN $J$ AND $K{\rm s}$ BANDS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.13732-b31b1b.svg)](https://arxiv.org/abs/2303.13732)<mark>Appeared on: 2023-03-27</mark> - __

</div>
<div id="authors">

Y. Shi, et al. -- incl., <mark>M. Zhai</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** Hot Jupiters, particularly those with temperature higher than 2000 K are the best sample of planets that allow in-depth characterization of their atmospheres. We present here a thermal emission study of the ultra hot Jupiter WASP $\mbox{-}$ 103 b observed in two secondary eclipses with CFHT/WIRCam in $J$ and $K_{\rm s}$ bands. By means of high precision differential photometry, we determine eclipse depths in $J$ and $K_{\rm s}$ to an accuracy of 220 and 270 ppm, which are combined with the published HST/WFC3 and Spitzer data to retrieve a joint constraints on the properties of WASP-103 b dayside atmosphere. We find that the atmosphere is best fit with a thermal inversion layer included. The equilibrium chemistry retrieval indicates an enhanced C/O (1.35 $^{+0.14}_{-0.17}$ ) and a super metallicity with [ Fe/H ] $=2.19^{+0.51}_{-0.63}$ composition. Given the near-solar metallicity of WASP-103 of [ Fe/H ] =0.06, this planet seems to be $\sim$ 100 more abundant than its host star. The free chemistry retrieval analysis yields a large abundance of FeH, H $^{-}$ , $CO_2$ and $CH_4$ . Additional data of better accuracy from future observations of JWST should provide better constraint of the atmospheric properties of WASP-103b.

</div>

<div id="div_fig1">

<img src="tmp_2303.13732/./pic/model_103b.png" alt="Fig7" width="100%"/>

**Figure 7. -** Observations and model spectra of dayside thermal emission of WASP-103b. Our $J$ and $K_{\rm s}$ data are presented by purple solid square, and other archival data are presented by black solid circle. The orange line shows the EQ model, while the violet line shows the FREE model. $\ww${The two dotted lines show the EQ and FREE models assuming isothermal. } (*Fig7*)

</div>
<div id="div_fig2">

<img src="tmp_2303.13732/./pic/rxb2_J.png" alt="Fig2" width="100%"/>

**Figure 2. -** The normalized rms$\times\beta^{2}$ distribution overlaid with contour maps for the various combinations of aperture diameter $D$ and the number of reference star group ($N_{\rm RSG}$) in the $J$ band. The minimum value of rms$\times\beta^{2}$ is reached with $D=34$ and $N_{\rm RSG}=5$. (*Fig2*)

</div>
<div id="div_fig3">

<img src="tmp_2303.13732/./pic/rms-beta.png" alt="Fig6" width="100%"/>

**Figure 6. -** RMS and noise factor $\beta$ of our residuals to the best-fit model for the various data sets in $J$ band.  (*Fig6*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.13732"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

175  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
